In [21]:
import os
from glob import glob

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd

import pumpp
import jams
import numpy as np

## IMPORTANT: define your working path

In [22]:
working = '/Users/manuel/working'

In [27]:
# LOAD TRAIN AND TEST IN THE PUMP
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

AUDIO = jams.util.find_with_extension(working+'/jazznet/clean_dataset/audios/', 'wav')
ANNOS = jams.util.find_with_extension(working+'/jazznet/clean_dataset/jams/', 'jams')

print(len(AUDIO))
print(len(ANNOS))

# Make sure there are the same number of files
assert len(AUDIO) == len(ANNOS)

# And that they're in agreement
assert all([root(_1) == root(_2) for (_1, _2) in zip(AUDIO, ANNOS)])

3144
3144


In [28]:
# Build a pump
sr = 44100
hop_length = 4096

p_feature = pumpp.feature.CQTMag(name='cqt', sr=sr, hop_length=hop_length, log=True, conv='tf', n_octaves=6)
p_chord_tag = pumpp.task.ChordTagTransformer(name='chord_tag', sr=sr, hop_length=hop_length, sparse=True)
p_chord_struct = pumpp.task.ChordTransformer(name='chord_struct', sr=sr, hop_length=hop_length, sparse=True)

pump = pumpp.Pump(p_feature, p_chord_tag, p_chord_struct)

# Save the pump

with open(working+'/chords/pump.pkl', 'wb') as fd:
    pickle.dump(pump, fd)

In [29]:
def convert(aud, jam, pump, outdir):
    
    data = pump.transform(aud, jam)
    
    fname = os.path.extsep.join([root(aud), 'npz'])
    
    np.savez(os.path.join(outdir, fname), **data)

In [30]:
OUTDIR = working+'/chords/pump/'

In [31]:
Parallel(n_jobs=20, verbose=10)(delayed(convert)(aud, jam, pump, OUTDIR) for (aud, jam) in zip(AUDIO, ANNOS));

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   10.8s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   11.6s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   12.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   12.8s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   13.3s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   13.9s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   14.4s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   15.0s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   15.8s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   16.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   16.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   17.4s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   18.0s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   18.5s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  